# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 5
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
train_paths = glob('../dataset/cifar/train/*.png')[:100]
test_paths = glob('../dataset/cifar/test/*.png')[:100]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

# Checkpoint

In [14]:
save_path = 'ckpt'

In [15]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(save_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

## Training

http://localhost:6006

In [16]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs, 
    callbacks=[checkpoint]
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
3/3 [==============================] - ETA: 0s - loss: 2.4774 - accuracy: 0.0294  
Epoch 00001: val_accuracy improved from -inf to 0.05208, saving model to ckpt
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ckpt\assets
3/3 [==============================] - 7s 2s/step - loss: 2.4774 - accuracy: 0.0294 - val_loss: 2.3329 - val_accuracy: 0.0521
Epoch 2/5
3/3 [==============================] - ETA: 0s - loss: 2.2940 - accuracy: 0.1458
Epoch 00002: val_accuracy improved from 0.05208 to 0.11458, saving model to ckpt
INFO:tensorflow:Assets written to: ckpt\assets
3/3 [==============================] - 7s 2s/step - loss: 2.2940 - accuracy: 0.1458 - val_loss: 2.3039 - val_accuracy: 0.1146
E

# Saving Model

In [17]:
save_path = 'my_model.h5'

In [18]:
# Save the model architecture and the weights
model.save(save_path, include_optimizer=True)

In [19]:
# Load the model architecture and the weights
model = tf.keras.models.load_model('my_model.h5')

# Saving Model - 2

In [20]:
# Save the weights
model.save_weights('model_weights.h5')

# Save the model architecture
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())

In [21]:
from tensorflow.keras.models import model_from_json

# Model reconstruction from JSON file
with open('model_architecture.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('model_weights.h5')

# model.h5 들여다보기

In [22]:
import h5py

model_file = h5py.File('my_model.h5','r+')    

In [23]:
model_file.keys()

<KeysViewHDF5 ['model_weights', 'optimizer_weights']>

In [24]:
model_file['model_weights'].keys()

<KeysViewHDF5 ['activation', 'activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1', 'dropout', 'dropout_1', 'dropout_2', 'flatten', 'input_1', 'max_pooling2d', 'max_pooling2d_1']>

In [25]:
model_file['model_weights']['conv2d']['conv2d'].keys()

<KeysViewHDF5 ['bias:0', 'kernel:0']>

In [26]:
model_file['model_weights']['conv2d']['conv2d']['kernel:0']

<HDF5 dataset "kernel:0": shape (3, 3, 3, 32), type "<f4">

In [27]:
# weights
np.array(model_file['model_weights']['conv2d']['conv2d']['kernel:0'])

array([[[[-4.53572646e-02, -5.29546551e-02, -4.70103137e-02,
          -2.26996746e-02, -1.19820349e-01, -1.90702975e-02,
          -6.38691336e-02,  1.15602724e-01, -1.36030912e-01,
           3.71177308e-02,  1.27868757e-01, -5.85133396e-02,
          -1.34819880e-01, -3.17175463e-02, -2.09058113e-02,
          -3.72474790e-02,  2.31237859e-02, -1.13544741e-03,
          -1.14658915e-01,  1.17151961e-01,  6.07927479e-02,
          -1.74229201e-02, -4.69665155e-02, -5.50862178e-02,
           1.90830845e-02,  1.08348876e-01, -6.69986606e-02,
          -2.67529693e-02,  1.29916951e-01, -6.03243187e-02,
           4.51796204e-02, -2.92180572e-02],
         [-1.24626748e-01, -1.20233782e-01, -3.13271806e-02,
          -8.92756283e-02, -9.98244882e-02,  6.56903535e-02,
           5.22852056e-02,  6.96137771e-02, -1.18468821e-01,
          -9.27053764e-02,  1.38334766e-01,  6.99368566e-02,
           9.13650021e-02, -7.35178366e-02,  3.28786857e-02,
          -1.87299470e-03, -2.37283465e-